# 数据集

## 数据集基础

In [14]:
import h5py
import numpy as np
f = h5py.File(r"D:\资料\HDF5和NC读取\HDF文件读取\testfile.hdf5","w")
arr = np.ones((5, 2))
f["my dataset"] = arr
dset = f["my dataset"]
dset

OSError: Unable to create file (unable to truncate a file which is already open)

In [15]:
# 读和写
## 读取数组
out1 = dset[:]
## 复制对象
out2 = dset

## 写 
dset[2,0:2] = 2
dset[:]
## Note:Dataset与numpy差别在于其在磁盘
## 上而不是内存，混用可能出现问题

array([[1., 1.],
       [1., 1.],
       [2., 2.],
       [1., 1.],
       [1., 1.]])

In [16]:
# 创建空数据集
dset = f.create_dataset("test5",(10,10))
dset
## HDF会分配写入数据需要的空间
dset = f.create_dataset("big dataset",(1024**3,),dtype=np.float32)
dset[0:1024] = np.arange(1024)
## f.flush()
## Note:flush后会写入磁盘（好像是）

In [17]:
# 指定储存类型节省空间
## 需写入的数据
bigdata = np.ones((100,1000))
bigdata.dtype

## 直接写入以及通过create_dataset的方法写入
with h5py.File("big1.hdf5", "w") as f1:
    f1["big"] = bigdata

with h5py.File("big2.hdf5", "w") as f2:
    f2.create_dataset("big", data = bigdata, dtype = np.float32)


In [18]:
# 自动类型转换与直读
## 对于大数组，最好的办法是直接读入
## 一个预分配的指定类型的 NumPy数组。
f2 = h5py.File("big2.hdf5")
dset = f2["big"]
big_out = np.empty((100, 1000), dtype = np.float64)
dset.read_direct(big_out)
big_out.dtype

<ipython-input-18-d35a1cd8ed5e>:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f2 = h5py.File("big2.hdf5")


dtype('float64')

In [19]:
# 用astype读
## 上面的方法也可以代替为这个
with dset.astype("float64"):
    out = dset[0,:]
out.dtype

dtype('float64')

In [20]:
# 改变形状
## 能够通过改变数组形状，对图片进行分区。

# 默认填充值
## 在create_dataset时进行使用
dset = f.create_dataset('empty', (2, 2), dtype = np.int32, fillvalue = 42)


## 读写数据

In [21]:
# 切片的效率问题
dset = f2["big"]
dset
out = dset[0:10, 20:70]
out.shape

# start-stop-step索引
## h5py不支持逆序


(10, 50)

In [22]:
# 多维切片和标量切片
## dset = f.create_dataset("4d", shape=(100, 80, 50, 20))
## dset[0,:,:,0].shape

## 标量数据集，两种特殊的数据
dset = f.create_dataset("1d", shape = (1,),data = 42)
## 数值
dset[0]
## 数组
dset[:]

dset2 = f.create_dataset("0d2", data = 42)
# 报错
dset[0]
# 数组
dset2[:]

ValueError: Illegal slicing argument for scalar dataspace

# 分块和压缩
## 分块存储
数据是连续储存的，而在实际应用中，可能会出现储存与实际不匹配的状况，所以我们需要分块存储。

In [11]:
dset = f.create_dataset("chunked", (100, 480, 640), dtype = "i1", chunks = (1,64,64))

In [13]:
dset.chunks

(1, 64, 64)

而且也可以进行自动分块

In [14]:
dset = f.create_dataset("Images",(100, 480, 640), "f", chunks = True)
dset.chunks
# 会尽可能保持n维的方形

(7, 30, 80)

因为每次读取是整块读取到内存，所以我们选择分块也要考虑限制因素
- 分块越少，读取越快，占用内存越多，大于1MB就不行了

## 过滤器与压缩
HDF5拥有一个过滤器流水线的概念，它实质上是在写入时对每个分块进行一系列的操作。每个过滤器都可以对分块中的数据做任何事情：压缩、计算校验和、添加元数据等任何事。文件被读取时，每个过滤器以反向模式运行来重新构建原始数据。


# 组、链接、迭代
## 组

In [26]:
# File也是Group的一个子类
f = h5py.File("D:\资料\HDF5和NC读取\HDF文件读取\Groups2.hdf5")
subgroup = f.create_group("SubGroup")

<ipython-input-26-66e7fef720ca>:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File("D:\资料\HDF5和NC读取\HDF文件读取\Groups2.hdf5")


In [27]:
subsubGroup = subgroup.create_group("AnotherGroup")
subsubGroup.name

'/SubGroup/AnotherGroup'

In [28]:
f["Dataset1"] = 1
f["Dataset2"] = 2
f["Dataset3"] = 3 
subgroup["Dataset4"] = 4


## 链接
### 硬链接
“硬”链接是将一个链接名字和文件中的对象关联到一起

In [29]:
f = h5py.File("linksdemo.hdf5","w")
grpx = f.create_group("x")
grpx.name

'/x'

In [30]:
f["y"] = grpx
grpy = f["y"]
grpy == grpx

True

### 软链接
“软”链接则是在对象内保存了指向一个对象的路径。因为只保存了路径，当我们重命名数据集并用别的东西取代它的位置时，softlink会指向新的对象。

In [23]:
f = h5py.File(r"D:\资料\HDF5和NC读取\HDF文件读取\test2.hdf5","w")
grp = f.create_group("mygroup")
dset = grp.create_dataset("dataset", (100,))


In [26]:
f["hardlink"] = dset
f["hardlink"] == grp["dataset"]
grp.move("dataset", "new_dataset_name")
f["hardlink"] == grp["new_dataset_name"]

True

In [32]:
grp.move("new_dataset_name", "dataset")
f["softlink"] = h5py.SoftLink("/mygroup/dataset")
f["softlink"] == grp["dataset"]

ValueError: Unable to move link (an object with that name already exists)

In [33]:
grp.move("dataset", "new_dataset_name")
dset2 = grp.create_dataset("dataset",(50,))
f["softlink"] == dset
f["softlink"] == dset2

ValueError: Unable to move link (an object with that name already exists)

### 外部链接
外部链接允许你指向别的文件内的对象。它因为是透明的，所以成为了HDF5最酷的功能之一，同时也是最难以追踪的问题之一。

In [32]:
with h5py.File("file_with_resource.hdf5","w") as f1:
    f1.create_group("mygroup")

In [33]:
f2 = h5py.File("linking_file.hdf5","w")
f2["linkname"] = h5py.ExternalLink("file_with_resource.hdf5","mygroup")

## 迭代与容器


In [34]:
[x for x in f]
[y for y in f.itervalues()]

AttributeError: 'File' object has no attribute 'itervalues'

In [40]:
group = f.create_group("testgroup2")

ValueError: Unable to create group (name already exists)

In [52]:
"name" in group

True